In [1]:
import pandas as pd
import random
from datetime import datetime, timedelta

# Function to generate random date within the specified range
def random_date(start_date, end_date):
    return start_date + timedelta(days=random.randint(0, (end_date - start_date).days))

# Function to generate unique random numbers
def generate_unique_random_numbers(n, start, end):
    numbers = set()
    while len(numbers) < n:
        numbers.add(random.randint(start, end))
    return list(numbers)

# Function to generate unique random strings
def generate_unique_random_strings(n, length):
    strings = set()
    while len(strings) < n:
        strings.add(''.join(random.choices('0123456789', k=length)))
    return list(strings)

# Function to generate individual dataset
def generate_dataset(project_amount, num_rows, legal_entity, line_of_business):
    # Transaction Date range
    start_date = datetime(2023, 1, 1)
    end_date = datetime(2023, 12, 31)

    # Account Category distribution
    num_capex = num_rows // 3
    num_opex = num_rows - num_capex

    capex_amounts = generate_unique_random_numbers(num_capex, 10000, 99999)
    opex_amounts = generate_unique_random_numbers(num_opex, 1000, min(30000, project_amount * 0.3))

    # Supplier Name and PO Number
    num_unique_po = num_rows // 2
    po_numbers = generate_unique_random_strings(num_unique_po, 5)
    ##supplier_names = random.sample(generate_unique_random_strings(num_rows, 8), num_unique_po)

    # [!!!!ADJUSTMENT!!!!!] Account Numbers
    account_numbers = []
    for cat in ['Capex'] * num_capex + ['Opex'] * num_opex:
        if cat == 'Capex':
            account_numbers.append(random.choice(['181030', '181032'])) #['181030', '181032']['165030', '165032']
        else:
            account_numbers.append('6' + ''.join(random.choices('0123456789', k=5)))

    #  [!!!!ADJUSTMENT!!!!!]Status
    statuses = ['Closed'] * num_rows

    # Legal Entity and Line of Business
    legal_entities = [legal_entity] * num_rows
    lines_of_business = [line_of_business] * num_rows

    # Project ID (all the same)
    project_ids = 'NL01-13'

    # Create DataFrame
    df = pd.DataFrame({
        'Transaction Date': [random_date(start_date, end_date).strftime('%d/%m/%y') for _ in range(num_rows)],
        'Account Category': ['Capex'] * num_capex + ['Opex'] * num_opex,
        'Amount': capex_amounts + opex_amounts,
        'Invoice': generate_unique_random_strings(num_rows, 4),
        'Voucher Number': generate_unique_random_strings(num_rows, 6),
        'PO Number': ['PO' + po for po in random.choices(po_numbers, k=num_rows)],
        ##'Supplier Name': random.choices(supplier_names, k=num_rows),
        'Status': statuses,
        'Account Number': account_numbers,
        'Legal Entity': legal_entities,
        'Line of Business': lines_of_business,
        'Project ID': project_ids
    })

    return df
###[!!!!ADJUSTMENT!!!!!]
# Define project amount and number of rows
project_amount = 20000
num_rows = 63
###[!!!!ADJUSTMENT!!!!!]
# Define Legal Entity and Line of Business
legal_entity = 'NL01'
line_of_business = '10100' #10001/10100

# Generate and save individual dataset
df = generate_dataset(project_amount, num_rows, legal_entity, line_of_business)

#Clean datetime 
df['Transaction Date'] = pd.to_datetime(df['Transaction Date'], format='mixed')

##Add supplier column
unique_po_numbers = df['PO Number'].unique()
# Predefined list of suppliers
suppliers = ['Supplier A', 'Supplier B', 'Supplier C', 'Supplier D', 'Supplier E', 'Supplier F', 'Supplier G']  # Update with your list of suppliers
# Create a dictionary to store supplier assignment for each PO Number
supplier_mapping = {}
# Assign suppliers to each unique PO Number in a cyclical manner
for i, po_number in enumerate(unique_po_numbers):
    supplier = suppliers[i % len(suppliers)]
    supplier_mapping[po_number] = supplier
# Create a new column 'Supplier' based on the assignment
df['Supplier'] = df['PO Number'].map(supplier_mapping)

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63 entries, 0 to 62
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Transaction Date  63 non-null     datetime64[ns]
 1   Account Category  63 non-null     object        
 2   Amount            63 non-null     int64         
 3   Invoice           63 non-null     object        
 4   Voucher Number    63 non-null     object        
 5   PO Number         63 non-null     object        
 6   Status            63 non-null     object        
 7   Account Number    63 non-null     object        
 8   Legal Entity      63 non-null     object        
 9   Line of Business  63 non-null     object        
 10  Project ID        63 non-null     object        
 11  Supplier          63 non-null     object        
dtypes: datetime64[ns](1), int64(1), object(10)
memory usage: 6.0+ KB


In [3]:
df.head()

,Transaction Date,Account Category,Amount,Invoice,Voucher Number,PO Number,Status,Account Number,Legal Entity,Line of Business,Project ID,Supplier
0,2023-06-14,Capex,50823,0797,622673,PO22564,Closed,181032,NL01,10100,NL01-13,Supplier A
1,2023-06-28,Capex,45068,2525,078647,PO46778,Closed,181030,NL01,10100,NL01-13,Supplier B
2,2023-10-09,Capex,40845,7221,841592,PO80257,Closed,181032,NL01,10100,NL01-13,Supplier C
3,2023-04-06,Capex,45965,6856,055521,PO28979,Closed,181030,NL01,10100,NL01-13,Supplier D
4,2023-10-16,Capex,59541,2098,988257,PO14646,Closed,181030,NL01,10100,NL01-13,Supplier E


In [4]:
df.isnull().sum()

Transaction Date    0
Account Category    0
Amount              0
Invoice             0
Voucher Number      0
PO Number           0
Status              0
Account Number      0
Legal Entity        0
Line of Business    0
Project ID          0
Supplier            0
dtype: int64

In [5]:
df.to_csv('project_13.csv', index=False)